In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
from typing import Optional, Dict, Sequence
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from copy import deepcopy
import copy
import logging
import json
import loralib as lora

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from dataclasses import dataclass
from transformers import AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy

In [2]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained=os.getenv('HOME')+'/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained=os.getenv('HOME')+'/aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/ko-gpt-trinity-1.2B-v0.5', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [3]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [4]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [5]:
with open(os.getenv('HOME')+'/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [6]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=2, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained(os.getenv('HOME')+'aiffel/KoChatGPT/output_3_PPO')

Train epoch [1/1]:  83%|████████▎ | 10/12 [00:05<00:01,  1.83it/s, actor_loss=-.649, critic_loss=0.189]


Episode [10/10]: 100%|██████████| 3/3 [01:09<00:00, 23.22s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   │   │   max_timesteps=3,                                                                │
│   21 │   │   │   update_timesteps=3)                                                             │
│   22                                                                                             │
│ ❱ 23 model.save_pretrained(os.getenv('HOME')+'aiffel/KoChatGPT/output_3_PPO')                    │
│   24                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [7]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'인입니다.\n\n1.\n그다: 물로 주문하거나 다양한 종류이므로 할 수 있습니다. 일반적으로 다음과 같습니다.\n- 휴이나 색제를 구매하는 옷에 주문해서 확인할 수 없습니다. 예를 들어, 일반적으로 영양이나 상체의 식사를 섭취해 먹을 수 있습니다. 예를 들어, 카페나 레스토랑이나 구매이나 음식 주문 서비스를 예약하고, 식료를 확인하고 구매할 수 있습니다. 따라서 예약, 여행나 예약, 주문을 선택하는 경우 주문 등을 선택할 수 있습니다. 호텔, 카드나 음식, 브랜드와 커피가 필요합니다.\n\n\n1. 구매와 결제업체, 인터넷 쇼핑몰으로 예약, 예약 예약 및 식시간을 확인하는 시간이나 스마트폰에 따라 다양한 가격을 찾아보니다.\n\n따한 카드을 찾아서 사용할 수 있습니다. 가능합니다. 만약 예세요, 레스토랑나 서비스: 세탁 예약해보세요. 상담 가능한 호텔에 예약에 집중해보세요. 예약장에서 구매하세요\n\n\n또시기, 예약하시면 예약 시간을 선택하는 것이 좋습니다. 주문하세요\n7. 상황에 맞게 체크: 음식 결제이나 온라인 여행에 맞는 결제이

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'저는 인공지능 어맥이므로, 현재 관련된 정보를 알 수 없습니다. \n이 어디한 지역의 질문에 대한 질문이 있는지 추가 정보가 불가능합니다. 추가적인 답변을 제공할 수 있습니다. 감사하신 정보를 알려 바랍으니, 해당 기사를 직접 질문해주시는 것이 좋습니다. 감사합니다. 감사합니다. 감사합니다. 감사합니다. 감사합니다. 감사합니다. 감사합니다. 혹시 감사한 정보는 자세한 기관나 공무장에서 등록자에게 직접 문의시길: 어떤 정보를 문의이 무엇이 어느 지역의 지역의 관련하여 구체적인 정보가 있다면 답변드리십시오. 감사시면 답변 관련 정보를 문의해으실 경우 관련 문의사항나 전화 관련 회사나 부동산 중개이나 관련 등록이